In [10]:
# This notebook is for training modified Chronos to classify activities

import numpy as np
from sklearn.metrics import confusion_matrix
from transformers import AutoConfig
import modified_chronos
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os

model_id = "amazon/chronos-bolt-base"
config = AutoConfig.from_pretrained(model_id)
NUM_CLASSES = 12

model = modified_chronos.ChronosBoltModelForClassification(
    config,
    num_classes=NUM_CLASSES,
    num_channels=2,
).to("cuda")

print("Model loaded successfully!")

for param in model.parameters():
    param.requires_grad = True

Model loaded successfully!


In [13]:
# HARTH dataset location is ./data/harth

class HARTHDataset(Dataset):
    def __init__(self, file_names):
        self.acc1 = []
        self.acc2 = []
        self.distribution = [0 for _ in range(NUM_CLASSES)]
        self.labels = []
        chunk_size = 150
        step_size = 75
        original_labels = [1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 130, 140]
        label_mapping = {old_label: new_label for new_label, old_label in enumerate(original_labels)}

        for file_name in file_names:
            print(f"Processing {file_name}")
            raw_path = os.path.join("data", "harth", file_name)
            df = pd.read_csv(raw_path)

            # Un-comment below to exclude cycling
            # df = df[df['label'] <= 8]

            # Adjust labels to be 0-indexed for CrossEntropyLoss
            df['label'] = df['label'].map(label_mapping)

            # Iterate through the DataFrame in chunks of 40
            for i in range(0, len(df) - chunk_size, step_size):
                chunk = df.iloc[i : i + chunk_size]

                # Ensure the chunk contains only one unique activity label
                if chunk['label'].nunique() == 1 and len(chunk) == chunk_size:
                    back_x = torch.tensor(chunk['back_x'].values, dtype=torch.float32)
                    back_y = torch.tensor(chunk['back_y'].values, dtype=torch.float32)
                    back_z = torch.tensor(chunk['back_z'].values, dtype=torch.float32)

                    thigh_x = torch.tensor(chunk['thigh_x'].values, dtype=torch.float32)
                    thigh_y = torch.tensor(chunk['thigh_y'].values, dtype=torch.float32)
                    thigh_z = torch.tensor(chunk['thigh_z'].values, dtype=torch.float32)

                    univariate = torch.concat([back_x, back_y, back_z])
                    self.acc1.append(univariate)
                    self.acc2.append(torch.concat([thigh_x, thigh_y, thigh_z]))
                    self.labels.append(chunk['label'].iloc[0])
                    self.distribution[chunk['label'].iloc[0]] += 1

    def __len__(self):
        return len(self.acc1)

    def __getitem__(self, idx):
        session = self.acc1[idx]
        session2 = self.acc2[idx]
        label = self.labels[idx]
        # 2 channel input
        return np.array([session, session2]), label

In [14]:
train_dataset = HARTHDataset(["S006.csv", "S009.csv", "S010.csv", "S012.csv", "S013.csv", "S014.csv", "S015.csv", "S020.csv", "S021.csv", "S022.csv", "S023.csv", "S024.csv", "S025.csv", "S026.csv", "S027.csv", "S028.csv", "S029.csv"])
test_dataset = HARTHDataset(["S008.csv", "S016.csv", "S017.csv", "S018.csv", "S019.csv"])

Processing S006.csv
Processing S009.csv
Processing S010.csv
Processing S012.csv
Processing S013.csv
Processing S014.csv
Processing S015.csv
Processing S020.csv
Processing S021.csv
Processing S022.csv
Processing S023.csv
Processing S024.csv
Processing S025.csv
Processing S026.csv
Processing S027.csv
Processing S028.csv
Processing S029.csv
Processing S008.csv
Processing S016.csv
Processing S017.csv
Processing S018.csv
Processing S019.csv


In [15]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [16]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

learning_rate = 0.0001
epochs = 40
class_counts = torch.tensor(train_dataset.distribution, dtype=torch.float)
print(train_dataset.distribution)
class_weights = 1. / class_counts
class_weights = class_weights / class_weights.sum() # Normalize
class_weights = class_weights.to(model.device) # Move weights to the correct device
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=15)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(model.device)
        y = y.to(model.device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    all_preds = []
    all_labels = []

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(model.device)
            y = y.to(model.device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            predicted_class = pred.argmax(1)
            correct += (predicted_class == y).type(torch.float).sum().item()
            all_preds.extend(predicted_class.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    cm = confusion_matrix(all_labels, all_preds)
    print(cm)
    return test_loss, correct


highest_accuracy = 0.0
best_model = model

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loss, accuracy = test_loop(test_dataloader, model, loss_fn)
    scheduler.step(test_loss)
    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        best_model = model
        print("Best model updated")
print("Done!")

[11529, 3423, 500, 459, 295, 5142, 24744, 3826, 4442, 565, 393, 72]
Epoch 1
-------------------------------
loss: 2.487115  [   64/55390]
loss: 2.488559  [ 6464/55390]
loss: 2.435345  [12864/55390]
loss: 2.424456  [19264/55390]
loss: 2.212585  [25664/55390]
loss: 2.453307  [32064/55390]
loss: 2.644957  [38464/55390]
loss: 1.869479  [44864/55390]
loss: 1.928174  [51264/55390]
Test Error: 
 Accuracy: 13.4%, Avg loss: 1.896113 

[[   41  1168     0     5     0     0     0     0   670    18     0     0]
 [    0   313     0     0     0     0     0     0     1     0     0     0]
 [    8    46     0     0     0     0     1     0   197     1     0     0]
 [    3    11     0     0     0     0     0     0   131     0     0     0]
 [    1    55     0     0     0     0     0     0    27     0     0     0]
 [   83   833     0     8     0     0     0     0   949    38     0     0]
 [   39   835     0     2     0     0   313 10177  2350     4     0     0]
 [    0     0     0     0     0     0    12  